In [7]:
import pandas as pd
import csv
import numpy as np
import math
from pyproj import CRS
from pyproj import Transformer

The data comprises two tables. 
1. school_id and its characteristics (religion, govermnental, student age...)
2. school_id and its XY coordinates

The first thing is to relate each school_id to the gush it belongs.
To do so, a post-request is sent to the website govmap with the XY coordinate of the scool_id in the payload.
The response contains the gush where the school is located.

In [8]:
def school_in_gush(x, y):
    request_payload = {"x":x,"y":y,"mapTolerance":100,
                       "layers":[{"LayerType":0,"LayerName":"school","LayerFilter":""},
                                 {"LayerType":0,"LayerName":"kids_g","LayerFilter":""},
                                 {"LayerType":0,"LayerName":"sub_gush_all","LayerFilter":""}]}

    headers = {'content-type': 'application/json'}
    url = r'https://ags.govmap.gov.il/Identify/IdentifyByXY'
    coordinate_request = requests.post(url, data=json.dumps(request_payload), headers=headers)
    school = coordinate_request.json()
    school_dict = {}
    school_in_gush = school['data'][1]['Result'][0]['tabs'][0]['fields'][0]['FieldValue']
    return school_in_gush


def list_school_in_gush(dataframe, output_path, column_label, start):
    new_df = pd.DataFrame(columns=column_label)
    file = open(output_path, "a")
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(column_label)
    x_coord = list(dataframe['X'])
    y_coord = list(dataframe['Y'])
    school_id = list(dataframe['school ID'])
    school_id = []
#     for tuples in list(dataframe.index):
#         school_id.append(tuples[0])
    print(school_id)
    gush_list = []
    # print(len(y_coord))
    for n in range(start, len(y_coord)):
        print('round: {}'.format(n))
        id = school_id[n]
        try:
            f_gush_number = school_in_gush(x=x_coord[n], y=y_coord[n])
            csv_row = ([id, f_gush_number])
            writer.writerow(csv_row)
        except:
            csv_row = ([id, 'none'])
            writer.writerow(csv_row)
        if n % 100 == 0:
            file.flush()


In [9]:
path1 = r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\SchoolCoordinate.csv'
output = r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\School and gush.csv'
output2 = r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\School_and_gushOK.csv'
df_school_coord = pd.read_csv(path1)
col_name = ['school ID', 'Gush number']
test_df_coord = df_school_coord.iloc[:20]
list_school_in_gush(dataframe=df_school_coord, output_path=output, column_label=col_name, start=0)

[]
round: 0


IndexError: list index out of range

Several of the results went back empty. To make sure that the problem does not come from an overloading of the website, the 'none' values were re-run for re-check.

In [10]:
df_school_gush = pd.read_csv(output)
none_gush = df_school_gush.loc[df_school_gush['Gush number'] == 'none']
none_gush = none_gush.replace('none', 'not found')
list_school_in_gush(dataframe=none_gush, output_path=output_none, column_label=col_name, start=0)


NameError: name 'output_none' is not defined

The results came back the same so we have to drop the school that are gush-less (404 entries).


Next, the school and gush will be merged with the school characteristics.

In [ ]:
df_school_gushOK = df_school_gush.set_index('Gush number')
df_school_gushOK.drop(index=df_school_gushOK.loc['none'])
df_school_gushOK.to_csv(output2)

In [ ]:
path1 = r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\SchoolCoordinate.csv'
path2 = r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\city_school.csv'
with open(path2, encoding='utf-8') as schoolspec:
    school_df = pd.read_csv(schoolspec)
    
to_be_drop = ['Name', 'number of student in the school', 'low status', 'mahoz','from level', 'to level','educ. To bagrut',
       'building year', 'founded','e', 'city.1','b', 'c', 'd', 'a', 'spcialized children', 'education type' ]

school_df.drop(columns = to_be_drop, inplace=True)
school_df

In [ ]:
school_df2 = school_df.set_index(['gan or highschool'], inplace=False)
# print(set((school_df2.index)))
school_df2.drop(index= ['מרכז חיר', 'מרכז ניה', 'פנימיה', 'מוסדות ל', 'מוסד תרב', 'הכשרות', 'מוסד ספו', '"היל""ה"', 'השלמת הש', 'אולפן'
                        , 'מסגרות ל',  'מרכז העש', 'סמינר', 'מכינה', 'אשכול'], inplace=True)

school_df2.reset_index(inplace=True)
school_df2.set_index('year', inplace=True)
school_df2.drop([2011, 2012, 2013, 2014], inplace=True)

school_df2

In [ ]:
output2 = r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\School_and_gushOK.csv'
df_school_gushOK = pd.read_csv(output2)
df_school_gushOK

In [ ]:
school = pd.merge(school_df2, df_school_gushOK, on='school_id', how='left')
# set(school_df2['gan or highschool'])
school

In [ ]:
gb_school = pd.pivot_table(school, index='Gush number', columns= ['gan or highschool'], values = 'school_id', aggfunc=np.count_nonzero)
gb_school.replace(np.nan,0, inplace=True)

gb_school['school_gan'] = gb_school['בית ספר'] + gb_school['גן ילדים']
gb_school['advanced_education'] = gb_school['אוניברסי'] + gb_school['מכללות']
gb_school.drop(columns = ['בית ספר', 'גן ילדים', 'אוניברסי', 'מכללות', 'advanced_education'], inplace=True)


gb_school2 = pd.pivot_table(school, index='Gush number', columns= ['legislation'], values = 'school_id', aggfunc=np.count_nonzero)
gb_school2.columns = ['Haredi_memchalti', 'memchalti', 'Haredi']


gb_school3 = pd.pivot_table(school, index='Gush number', columns= ['religion'], values = 'school_id', aggfunc=np.count_nonzero)
gb_school3.columns = ['beduin', 'druse', 'jewish', 'arabic', 'chersk']

gush_and_schooltype = pd.merge(gb_school, gb_school2, on='Gush number', how='left')
gush_and_schooltype = pd.merge(gush_and_schooltype, gb_school3, on='Gush number', how='left')
gush_and_schooltype.replace(np.nan,0, inplace=True)
gush_and_schooltype

Now to transform haredi, haredi memchalti and memchalti to gush with majority of school:

Haredi ==> H

Haredi memchalti ==> HM

memchalti ==> M

Haredi + Haredi memchalti ==> H

Else ==> mixed

And to replace ethnic majority:
jewish = J
Arabic = A
Other = O

In [ ]:
gush_and_schooltype.replace(np.nan,0, inplace=True)
education_type = []
for row, value in gush_and_schooltype.iterrows():
#     print(row)
    if value['Haredi_memchalti'] + value['Haredi'] > value['school_gan']/2:
        education_type.append('H')
    elif value['Haredi_memchalti']> value['school_gan']/2:
        education_type.append('HM')
    elif value['Haredi']> value['school_gan']/2:
        education_type.append('H')
    elif value['memchalti']> value['school_gan']/2:
        education_type.append('M')
    else:
        education_type.append('mixed')
# print(education_type)
#     print(gush_and_schooltype.loc[row, value['Haredi_memchalti']])
gush_and_schooltype['education_type'] = education_type

religion = []
for row, value in gush_and_schooltype.iterrows():
#     print(row)
    if value['jewish'] > value['school_gan']/2:
        religion.append('J')
    elif value['arabic']> value['school_gan']/2:
        religion.append('A')
    else:
        religion.append('O')
# print(education_type)
#     print(gush_and_schooltype.loc[row, value['Haredi_memchalti']])
gush_and_schooltype['ethnicity'] = religion

new_gush_and_schooltype = gush_and_schooltype.drop(columns = ['Haredi_memchalti', 'memchalti', 'Haredi','beduin', 'druse', 'jewish', 'arabic', 'chersk'])
new_gush_and_schooltype

# output =r'C:\Users\majac\Desktop\JohnBriceProject\SchoolData\school_education_in_gush.csv'
# new_gush_and_schooltype.to_csv(output)

In [45]:
file_name3 = r'C:\Users\majac\Desktop\JohnBriceProject\intermediate csv files\school_education_in_gush.csv'
gush_school = pd.read_csv(file_name3)
gush_school.set_index('gush_number', inplace=True)
gush_school.loc[309]

school_gan        10
education_type     H
ethnicity          J
Name: 309, dtype: object

In [48]:
for row, value in gush_school.iterrows():
    if gush_school.loc[row, 'ethnicity']== 'A':
        gush_school.loc[row, 'education_type']= 'A'
    if gush_school.loc[row, 'ethnicity']== 'O':
        gush_school.loc[row, 'education_type']= 'O'
        
gush_school['religious or not'] = gush_school['education_type'].replace({'H':4,'mixed':3, 'M':2, 'A':1, 'O':0})
print(set(gush_school['education_type']))
gush_school
# gush_school['education_type'].astype(int)
gush_schoolGB = gush_school.groupby('gush_number').sum()
# gush_schoolGB['religious education level'] = gush_schoolGB['religious or not']/gush_schoolGB['school_gan']
gush_school.to_csv(r'C:\Users\majac\Desktop\JohnBriceProject\intermediate csv files\school_education_in_gush2.csv')


{'mixed', 'H', 'A', 'M', 'O'}
